## Pixel that changes from 0-1 based on pattern

In [22]:
from typing import List, Tuple

In [19]:
import numpy as np

MIN_BASE_SWITCH = .10
MAX_BASE_SWITCH = .30
MIN_ADD_SWITCH = .05
MAX_ADD_SWITCH = .10

In [24]:
def simulate_pixel_flip_stochastic(timesteps: int, rng: np.random.Generator = None
                   ) -> Tuple[List[int], List[float]]:
    """
    Simulate a single pixel flipping between 0 and 1.

    Returns
    -------
    states : list[int]
        State at each timestep (length = `timesteps`)
    probs  : list[float]
        Switching probability *used* on each timestep
        (same length; values in 0–1)
    """
    if rng is None:  # allows deterministic tests
        rng = np.random.default_rng()

    # initial state and baseline switch probability
    state: int = rng.integers(0, 2)           # 0 or 1
    switch_p: float = (
        rng.uniform(MIN_BASE_SWITCH, MAX_BASE_SWITCH)
    )
    add_inc: float = (
        rng.uniform(MIN_ADD_SWITCH, MAX_ADD_SWITCH)
    )

    states: List[int] = []
    probs:  List[float] = []

    for _ in range(timesteps):
        # record current values *before* potential flip
        states.append(state)
        probs.append(switch_p)

        # decide whether to flip
        if rng.random() < switch_p:
            # flip the pixel
            state = 1 - state
            # reset baseline switching probability
            switch_p = (
                rng.uniform(MIN_BASE_SWITCH, MAX_BASE_SWITCH)
            )
            # draw a fresh additive increment for this new dwell
            add_inc = (
                rng.uniform(MIN_ADD_SWITCH, MAX_ADD_SWITCH)
            )
        else:
            # stayed in same state → increase hazard
            switch_p = min(1.0, switch_p + add_inc)

    return states, probs

In [24]:
def simulate_pixel_flip_deterministic(timesteps: int, rng: np.random.Generator = None
                   ) -> Tuple[List[int], List[float]]:
    """
    Simulate a single pixel flipping between 0 and 1.

    Returns
    -------
    states : list[int]
        State at each timestep (length = `timesteps`)
    probs  : list[float]
        Switching probability *used* on each timestep
        (same length; values in 0–1)
    """
    if rng is None:  # allows deterministic tests
        rng = np.random.default_rng()

    # initial state and baseline switch probability
    state: int = rng.integers(0, 2)           # 0 or 1
    switch_p: float = (MIN_BASE_SWITCH + MAX_BASE_SWITCH) / 2
    add_inc: float = (MIN_ADD_SWITCH + MAX_ADD_SWITCH) / 2

    states: List[int] = []
    probs:  List[float] = []

    for _ in range(timesteps):
        # record current values *before* potential flip
        states.append(state)
        probs.append(switch_p)

        # decide whether to flip
        if rng.random() < switch_p:
            # flip the pixel
            state = 1 - state
            # reset baseline switching probability
            switch_p = (MIN_BASE_SWITCH + MAX_BASE_SWITCH) / 2
            # draw a fresh additive increment for this new dwell
            add_inc = (MIN_ADD_SWITCH + MAX_ADD_SWITCH) / 2
        else:
            # stayed in same state → increase hazard
            switch_p = min(1.0, switch_p + add_inc)

    return states, probs

In [25]:
s, p = simulate_pixel(30)
for t, (st, pr) in enumerate(zip(s, p)):
    print(f"t={t:02d}  state={st}  switch_p={pr:.2f}")

t=00  state=1  switch_p=0.20
t=01  state=1  switch_p=0.26
t=02  state=1  switch_p=0.31
t=03  state=0  switch_p=0.13
t=04  state=0  switch_p=0.21
t=05  state=0  switch_p=0.29
t=06  state=0  switch_p=0.36
t=07  state=1  switch_p=0.28
t=08  state=1  switch_p=0.33
t=09  state=1  switch_p=0.39
t=10  state=0  switch_p=0.17
t=11  state=0  switch_p=0.23
t=12  state=0  switch_p=0.28
t=13  state=0  switch_p=0.34
t=14  state=0  switch_p=0.40
t=15  state=1  switch_p=0.26
t=16  state=1  switch_p=0.34
t=17  state=0  switch_p=0.13
t=18  state=0  switch_p=0.20
t=19  state=0  switch_p=0.27
t=20  state=0  switch_p=0.33
t=21  state=0  switch_p=0.40
t=22  state=1  switch_p=0.25
t=23  state=0  switch_p=0.18
t=24  state=1  switch_p=0.14
t=25  state=1  switch_p=0.22
t=26  state=1  switch_p=0.30
t=27  state=1  switch_p=0.38
t=28  state=1  switch_p=0.46
t=29  state=0  switch_p=0.19


**note**: if input is 1px and output is 1px, must retain an internal representation of what happened
- need either:
  - exact memory
  - noisy memory/recurrence 
  
**alt**: feed in a sequence of input (last `n` timesteps)

## TODO
- more complex patterns (e.g. if back and forth switch happens with `x` seconds, change some properties)
- 

# Continuous (or near) changes
- Switching codition
    - dynamics are static
    - dynamics are in flux
- come up with an algo/architecture that stays up to date

## Add small state space where you can take actions
3 squares
- moving costs 1
- being on the left when pixel turns to 0 yields +10, and right/1 yields +10
- being on the wrong side during change dings you -5

### Scenario
- pixel is always visible
- pixel is only visible from middle square
- pixel is never visible

## two pixel
- IID
- joint distribution
- extend to a string of pixels